In [1]:
from __future__ import annotations
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import argparse
import pandas as pd

from pathlib import Path
import os, sys

from fastai.data.all import Transform, DataBlock, RandomSplitter, L
from fastai.vision.all import *
from fastai.distributed import *
from fastai.callback.all import SaveModelCallback, EarlyStoppingCallback
import segmentation_models_pytorch as smp

# Load StarCraft2Sensor stuff
ipynb_dir = os.path.dirname(os.path.realpath("__file__"))
code_root = os.path.join(ipynb_dir, '..')
sys.path.append(code_root)  # Needed for import below

from sc2sensor.dataset import StarCraftSensor
from sc2sensor.utils.sensor_utils import SensorPlacementDataset, SUPPORTED_PLACEMENT_KINDS
from sc2sensor.utils.unit_type_data import NONNEUTRAL_CHANNEL_TO_ID, NONNEUTRAL_ID_TO_NAME
CHANNEL_TO_NAME = [NONNEUTRAL_ID_TO_NAME[NONNEUTRAL_CHANNEL_TO_ID[i]] for i in range(len(NONNEUTRAL_CHANNEL_TO_ID))]
# removing barrier_h and barrier_v from the placement kinds since diag kinds are more fitting
PLACEMENT_KINDS = [kind for kind in SUPPORTED_PLACEMENT_KINDS if not (kind.endswith('_h') or kind.endswith('_v'))]


data_root = os.path.join(code_root, 'data') # Data root directory
data_subdir = 'starcraft-sensor-dataset'

In [2]:
# extracted_model_root = Path('/local/scratch/a/skulinsk/starcraft-dataset-extraction-private/models/extracted_models')
# benchmark_results_save_root = Path('/local/scratch/a/skulinsk/starcraft-dataset-extraction-private/paper-demos/benchmark-results')

extracted_model_root = Path('/local/scratch/a/shared/starcraft_shared/benchmarks/benchmark-models')
benchmark_results_save_root = Path('/local/scratch/a/shared/starcraft_shared/benchmarks/v2-benchmark-results')

In [3]:
!ls $benchmark_results_save_root

In [4]:
# !rm $benchmark_results_save_root/*.csv 

In [5]:
class FakeArgs():
    def __str__(self):
        return str(self.__dict__)

args = FakeArgs()
args.max_samples = 10000
args.batch_size = 24
args.n_epochs = 10
args.max_models = 10000 # Set to 1 if you want to do a quick test run

print(f'Starting with inputs: {str(args)}')

if args.max_samples is not None:
    print(f'Using a max number of a {args.max_samples}')

Starting with inputs: {'max_samples': 10000, 'batch_size': 24, 'n_epochs': 10, 'max_models': 10000}
Using a max number of a 10000


In [6]:
def save_benchmark_results(save_root, result, name, benchmark_kind,
                             sensor_kwargs=None, metric_name='MSE', raw_model_path=None):
    if benchmark_kind == 'unit_identification':
        results_file = 'unit_identification_results.csv' if sensor_kwargs is None else 'sensor_unit_identification_results.csv'
    elif benchmark_kind == 'next_window':
        results_file = 'next_window_results.csv' if sensor_kwargs is None else 'sensor_next_window_results.csv'
    else:
        raise NotImplementedError(f'No benchmark file exists for type {benchmark_kind}')
    print(f'Appending to results file: {results_file}')
        
    save_root = Path(save_root)
    if (save_root/results_file).exists():
        current_results = pd.read_csv(save_root/results_file)
    else:
        current_results = pd.DataFrame()
        
    if sensor_kwargs is None:
      sensor_kwargs = {'n_sensors': 1, 'radius': 1000, 'kind': 'oracle', 'failure_rate': 0}
        
    results_to_be_saved = {
        'model_name':name,
         #metric_name: [result],
        # result is now a dictionary of metrics
        **result,
        'sensor_kind':sensor_kwargs['kind'],
        **sensor_kwargs,
        'sensor_kwargs': [sensor_kwargs],
        'raw_model_path':str(raw_model_path) if isinstance(raw_model_path, Path) else raw_model_path}
    
    
    results_df = pd.concat((current_results, pd.DataFrame(results_to_be_saved)))
    results_df.to_csv(save_root/results_file, index=False)
    return True

def get_best_model(model_root, with_suffix=False):
    model_root = Path(model_root)
    train_history = pd.read_csv(model_root/'train_history.csv')
    best_epoch = train_history.valid_loss.argmin()
    best_model = [i for i in list(model_root.glob('*')) if str(i).endswith(f'_{best_epoch}.pth')]
    assert len(best_model) == 1, f'Found {len(best_model)} best model(s) instead of just 1.'
    if with_suffix:
        return best_model[0]
    else:
        return best_model[0].with_suffix('')

# Next Window Experiments

In [7]:
# Create subclass of original dataset for next window prediction
class NextWindowDataset(StarCraftSensor):
  def __init__(self, *args, max_samples=None, **kwargs):
    assert 'use_sparse' not in kwargs, 'Cannot set use_sparse with this dataset.'
    assert 'compute_labels' not in kwargs, 'Cannot set use_sparse with this dataset.'
    super().__init__(*args, use_sparse=True, compute_labels=False, **kwargs)
    self.max_samples = max_samples
    
    # Sort data so that next index is merely + 1
    self.metadata = self.metadata.sort_values(['static.replay_name', 'dynamic.window_idx']).reset_index(drop=True)
    md = self.metadata
    
    # Get starting window indices
    start_windows = md[(md['dynamic.num_windows'] > 1) 
                       & (md['dynamic.window_idx'] < (md['dynamic.num_windows'] - 1))]
    self.start_idx = start_windows.index

  def __getitem__(self, idx):
    # Get original indices of start and end based on input 
    md = self.metadata
    # Assumes sorted
    orig_idx = self.start_idx[idx]
    next_idx = orig_idx + 1
    # Only sanity check first and last as this may be expensive
    if idx == 0 or idx == len(self) - 1:
      assert md['static.replay_name'][orig_idx] == md['static.replay_name'][next_idx], 'Replays are not the same'
      assert md['dynamic.window_idx'][orig_idx] + 1 == md['dynamic.window_idx'][next_idx], 'Window indices are not adjacent'

    # Get combined hyperspectral images
    def get_hyperspectral_dense(idx):
      # Concatenate player1 and player2 hyperspectral
      replay_file, window_idx = self._get_replay_and_window_idx(idx)
      with np.load(replay_file) as data:
        player_1_hyperspectral = self._extract_hyperspectral(
          'player_1', data, window_idx)
        player_2_hyperspectral = self._extract_hyperspectral(
          'player_2', data, window_idx)
      return torch.concat([player_1_hyperspectral.to_dense(), 
                           player_2_hyperspectral.to_dense()], 
                          dim=-3).float()
    windows = [get_hyperspectral_dense(idx) for idx in [orig_idx, next_idx]] 
    x = windows[0]
    y = windows[1] - windows[0] # Compute diff
    return x, y

  def __len__(self):
    if self.max_samples is not None:
      return min(self.max_samples, len(self.start_idx))
    else:
      return len(self.start_idx)

# Create fastai dataloaders given the PyTorch dataset
class AddChannelCodes(Transform):
  "Add the code metadata to a `TensorMask`"
  def __init__(self, codes=None):
      self.codes = codes
      if codes is not None: self.vocab,self.c = codes,len(codes)

  def decodes(self, o):
      if self.codes is not None: o.codes=self.codes
      return o
    
# HACK: Put all instances in both "train" and "valid"
# From https://forums.fast.ai/t/solved-not-splitting-datablock/84759/3
def all_splitter(o): return L(int(i) for i in range(len(o))), L(int(i) for i in range(len(o)))
    
SC2_CODES = [
  f'{player}_{name}'
  for player in ['P1','P2']
  for name in CHANNEL_TO_NAME
]
def create_dataloaders_from_dataset(dataset, splitter=None, **kwargs):
    # Needs to have reference to dataset for closures
    assert 'get_x' not in kwargs
    assert 'get_y' not in kwargs
    assert 'get_items' not in kwargs
    splitter = splitter if splitter is not None else RandomSplitter(seed=0)
    assert len(SC2_CODES) == dataset[0][0].shape[0], 'Number of codes does not match number of channels'
    block = DataBlock(
        get_items=lambda d: list(range(len(d))),
        get_x=lambda idx: dataset[idx][0],
        get_y=lambda idx: dataset[idx][1],
        blocks=None, # These are just transforms
        splitter=splitter,
        item_tfms=[AddChannelCodes(SC2_CODES)],
    )
    return block.dataloaders(dataset, **kwargs)

In [8]:
# Load test set
next_window_test = NextWindowDataset(root=data_root, subdir=data_subdir,
                                     train=False, max_samples=args.max_samples)
# We should not clip the test set
#, max_samples=args.max_samples)

print(f'Num Test: {len(next_window_test)}')
print(f'x.shape: {next_window_test[-1][0].shape}, y.shape: {next_window_test[-1][1].shape}')

Using cached CSV metadata
Not computing labels
Post-processing metadata
Finished dataset init
Num Test: 10000
x.shape: torch.Size([340, 64, 64]), y.shape: torch.Size([340, 64, 64])


In [9]:
# Create metrics
from fastai.metrics import AvgMetric, Metric
#from sklearn.metrics import confusion_matrix

class NumWindowsMetric(Metric):
  def reset(self):           self.count = 0
  def accumulate(self, learn):
    bs = find_bs(learn.yb)
    #print(self.total)
    self.count += bs
  @property
  def value(self): return self.count if self.count != 0 else None
  @property
  def name(self):  return 'NumWindows' 

# Example that takes an average (while considering batch size `bs`)
# %% ../nbs/13a_learner.ipynb 119
class BinnedConfusionMatrixMetric(Metric):
  """Micro confusion matrix (enables computation of micro-versions of other metrics)
     (i.e., will give total percentages over all batches)
  """
  def __init__(self):  self.labels = [-1.0, 1.0, 0.0]
  def reset(self):           self.total,self.count = 0.0,0
  def accumulate(self, learn):
    bs = find_bs(learn.yb)
    # Scales by batch size (later divided by batch size)
    self.total += learn.to_detach(self._metric_batch(learn.pred, *learn.yb)) * bs
    #print(self.total)
    self.count += bs
  def _metric_batch(self, y_pred, y_true):
    # Computes confusion matrix for this batch (Normalized to percentages)
    # First convert to 1, 0, or -1
    def bin_data(y):
      near_zero_mask = y.abs().lt(1/255.0)
      y = torch.sign(y)
      y[near_zero_mask] = 0
      return y
    y_pred, y_true = bin_data(y_pred), bin_data(y_true)
    labels = self.labels
    # Sklearn confusion_matrix is far too slow, vectorized version below
    cm = np.zeros((len(labels), len(labels)))
    for i, lab_i in enumerate(labels):
      for j, lab_j in enumerate(labels):
        cm[i, j] = torch.logical_and(y_true == lab_i, y_pred == lab_j).float().mean().cpu().numpy()
    return cm

  @property
  def value(self): return self.total/self.count if self.count != 0 else None
  @property
  def name(self):  return 'BinnedConfusionMatrix' 
  
class BinnedNonZeroFalsePositiveRate(BinnedConfusionMatrixMetric):
  """Micro FPR where 0 is negative - (True zeros but predicted nonzero)/(Num true zeros)"""
  @property
  def value(self):
    print('FPR calculation')
    micro_cm = self.total / self.count
    zero_idx = self.labels.index(0.0)
    correctly_predicted_zeros = micro_cm[zero_idx, zero_idx] # TN
    total_true_zeros = micro_cm[zero_idx, :].sum() # N
    TNR = correctly_predicted_zeros/total_true_zeros # TN/N
    # FPR = 1 - TNR
    return 1 - TNR if self.count != 0 else None
  @property
  def name(self):  return 'FPR' 
  
class BinnedTruePositiveRate(BinnedConfusionMatrixMetric):
  """Micro FPR where 0 is negative - (True zeros but predicted nonzero)/(Num true zeros)"""
  def __init__(self, positive_value):
    super().__init__()
    self.positive_value = positive_value
  @property
  def value(self):
    micro_cm = self.total / self.count
    positive_idx = self.labels.index(self.positive_value)
    correctly_predicted = micro_cm[positive_idx, positive_idx] # TP
    total_true = micro_cm[positive_idx, :].sum() # P
    TPR = correctly_predicted/total_true # TP/P
    return TPR if self.count != 0 else None
  @property
  def name(self):  return f'TPR({self.positive_value:.0f})'
  
next_window_metrics = [
  nn.MSELoss(), 
  BinnedNonZeroFalsePositiveRate(), 
  BinnedTruePositiveRate(1.0), 
  BinnedTruePositiveRate(-1.0)
]
next_window_metric_names = [m.name if hasattr(m, 'name') else 'MSE' for m in next_window_metrics]
print(next_window_metrics)
print(next_window_metric_names)

[MSELoss(), <__main__.BinnedNonZeroFalsePositiveRate object at 0x7f296ece5690>, <__main__.BinnedTruePositiveRate object at 0x7f296ece5710>, <__main__.BinnedTruePositiveRate object at 0x7f296ece57d0>]
['MSE', 'FPR', 'TPR(1)', 'TPR(-1)']


In [10]:
def test_next_window_model(arch, batch_size, path_to_model,
                           next_window_dataset, sensor_kwargs=None):

    if sensor_kwargs is not None:
        # build dataset with specific sensor type
        assert all(('n_sensors' in sensor_kwargs,
                   'radius' in sensor_kwargs,
                   'kind' in sensor_kwargs,
                   'failure_rate' in sensor_kwargs)) 
        next_window_dataset = SensorPlacementDataset(next_window_dataset, **sensor_kwargs,
                                                     return_mask=False, make_cache=False,
                                                     noiseless_ground_truth=True)
        
    dls = create_dataloaders_from_dataset(next_window_dataset, splitter=all_splitter, batch_size=batch_size)
    print(f'Test dataset has {len(next_window_dataset)} windows and the valid dataloader has approximately {len(dls.valid)*batch_size} windows')

    model = smp.Unet(
        encoder_name=arch,        
        encoder_weights='imagenet',     
        in_channels=len(SC2_CODES),                  
        classes=len(SC2_CODES),                      
        activation=None, 
    )
    
    learner = Learner(model=model, dls=dls, loss_func=nn.MSELoss())
    learner.metrics = next_window_metrics
    learner = learner.load(path_to_model)
    
#     with learner.distrib_ctx():
    result = learner.validate()
    # Now that training has finished, empty the cache
    print('Number of results vs number of names (I think original loss is automatically included)')
    print(len(result), len(next_window_metric_names))
    torch.cuda.empty_cache()
    result_dict = {m: r for r, m in zip(result[1:], next_window_metric_names)}
    print(result_dict)
    return result_dict

## Gathering next hyperspectral window results  (no sensor)

In [ ]:
%pdb on
# Next window experiments  (not using sensors)
next_window_model_root = extracted_model_root / 'next_window'
architectures = (
  'resnet18',
  'resnet34',
  'resnet50',
  'resnext50_32x4d',
)

experiments = [*architectures]

print(f'Starting testing on next window dataset. [NOTE: Not using sensors]',
      f'Testing with \n{experiments}')


for arch in experiments[:min(args.max_models,len(experiments))]:
    name = 'unet_' + arch
    print('\n'*5, f'Starting {name}.'.center(60, '-'))

    path_to_model = get_best_model(next_window_model_root / name)
    print(f'Using model {path_to_model}')
    
    result = test_next_window_model(arch, args.batch_size, path_to_model, next_window_test, None)
    
    save_benchmark_results(benchmark_results_save_root, result, name, benchmark_kind='next_window',
                             sensor_kwargs=None, metric_name='MSE', raw_model_path=path_to_model)

    print(f'Finished {name}placement.')

Automatic pdb calling has been turned ON
Starting testing on next window dataset. [NOTE: Not using sensors] Testing with 
['resnet18', 'resnet34', 'resnet50', 'resnext50_32x4d']





 ------------------Starting unet_resnet18.-------------------
Using model /local/scratch/a/shared/starcraft_shared/benchmarks/benchmark-models/next_window/unet_resnet18/next_window_60000_unet_resnet18_7
Test dataset has 10000 windows and the valid dataloader has approximately 10008 windows


FPR calculation
Number of results vs number of names (I think original loss is automatically included)
5 4
{'MSE': 3.85090708732605, 'FPR': 0.15030505040575948, 'TPR(1)': 0.6058672497798525, 'TPR(-1)': 0.5558459095472176}
Appending to results file: next_window_results.csv
Finished unet_resnet18placement.





 ------------------Starting unet_resnet34.-------------------
Using model /local/scratch/a/shared/starcraft_shared/benchmarks/benchmark-models/next_window/unet_resnet34/next_window_60000_unet_resnet34_8
Test dataset has 10000 windows and the valid dataloader has approximately 10008 windows


In [ ]:
pd.read_csv(benchmark_results_save_root/'next_window_results.csv')

## Gathering *sensor* next hyperspectral window results

In [ ]:
# Next window experiments  (not using sensors)
sensor_next_window_model_root = extracted_model_root / 'sensor_next_window'

sensor_next_window_model_dirs = [f for f in sensor_next_window_model_root.glob('**/*') 
                                   if f.is_dir() and not str(f).endswith('_sensors')]

print(f'Starting testing on sensor next window dataset.')


for model_dir in sensor_next_window_model_dirs[:min(args.max_models,len(sensor_next_window_model_dirs))]:
    sensor_kind = model_dir.parent.name.split('_sensors')[0]
    name = model_dir.name
    arch = name.split('unet_')[-1]
    
    sensor_kwargs = {'n_sensors': 50,
                     'radius': 5.5,
                     'kind': sensor_kind,
                     'failure_rate': 0.2}
    
    print('\n'*5, f'Starting {name} with {sensor_kind} placement.'.center(60, '-'))

    path_to_model = get_best_model(model_dir)
    
    print(f'Using model {path_to_model}')
    
    result = test_next_window_model(arch, args.batch_size, path_to_model, next_window_test, sensor_kwargs)
    
    save_benchmark_results(benchmark_results_save_root, result, name, benchmark_kind='next_window',
                             sensor_kwargs=sensor_kwargs, metric_name='MSE', raw_model_path=path_to_model)

    print(f'Finished {name} with {sensor_kind} placement.')

In [ ]:
pd.read_csv(benchmark_results_save_root/'sensor_next_window_results.csv')

# Unit Type Identification Experiments

In [ ]:
from sc2sensor.utils.unit_type_data import NONNEUTRAL_CHANNEL_TO_ID, NONNEUTRAL_ID_TO_NAME
CHANNEL_TO_NAME = [NONNEUTRAL_ID_TO_NAME[NONNEUTRAL_CHANNEL_TO_ID[i]] for i in range(len(NONNEUTRAL_CHANNEL_TO_ID))]

In [ ]:
# Loading dataset
class SegmentationDataset(torch.utils.data.Dataset):
    
    def __init__(self, segment_path, create_metadata=True):
        super().__init__()
        self.path = Path(segment_path)
        self.X_filenames = self._get_files()
        if create_metadata:
            self.metadata, self.match_metadata = self._make_metadata()
        
    def __len__(self):
        return len(self.X_filenames)
    
    def __getitem__(self, idx):
        X_filename = self.X_filenames[idx]
        y_filename = os.path.splitext(X_filename)[0].replace('images','labels') + '_labels.png'
        
#         return torch_read_image(str(X_filename)), torch_read_image(str(y_filename)).squeeze()
        return (plt.imread(str(X_filename))*255).astype(np.uint8), \
               (plt.imread(str(y_filename))*255).astype(np.uint8)

    def _get_files(self):
        files =  list((self.path / 'images').glob('**/*.png'))
        assert len(files) > 0, f'No .png files found in {self.path}'
        return files
    
    def _make_metadata(self):
        replay_names = [str(f).split('_')[-2].split('/')[-1] for f in self.X_filenames]
        metadata = pd.DataFrame({'static.replay_name':replay_names})
        match_metadata = metadata.drop_duplicates(subset=['static.replay_name']).reset_index(drop=True)
        return metadata, match_metadata

In [ ]:
segmentation_data_path = Path(data_root)/'starcraft-sensor-dataset'/'segment'/'test'

In [ ]:
# Create metrics
from fastai.metrics import AvgMetric, Metric
#from sklearn.metrics import confusion_matrix

class NumWindowsMetric(Metric):
  def reset(self):           self.count = 0
  def accumulate(self, learn):
    bs = find_bs(learn.yb)
    #print(self.total)
    self.count += bs
  @property
  def value(self): return self.count if self.count != 0 else None
  @property
  def name(self):  return 'NumWindows' 

# Example that takes an average (while considering batch size `bs`)
# %% ../nbs/13a_learner.ipynb 119
class BinnedConfusionMatrixMetric(Metric):
  """Micro confusion matrix (enables computation of micro-versions of other metrics)
     (i.e., will give total percentages over all batches)
  """
  def __init__(self):  self.labels = [-1.0, 1.0, 0.0]
  def reset(self):           self.total,self.count = 0.0,0
  def accumulate(self, learn):
    bs = find_bs(learn.yb)
    # Scales by batch size (later divided by batch size)
    self.total += learn.to_detach(self._metric_batch(learn.pred, *learn.yb)) * bs
    #print(self.total)
    self.count += bs
  def _metric_batch(self, y_pred, y_true):
    # Computes confusion matrix for this batch (Normalized to percentages)
    # First convert to 1, 0, or -1
    def bin_data(y):
      near_zero_mask = y.abs().lt(1/255.0)
      y = torch.sign(y)
      y[near_zero_mask] = 0
      return y
    y_pred, y_true = bin_data(y_pred), bin_data(y_true)
    labels = self.labels
    # Sklearn confusion_matrix is far too slow, vectorized version below
    cm = np.zeros((len(labels), len(labels)))
    for i, lab_i in enumerate(labels):
      for j, lab_j in enumerate(labels):
        cm[i, j] = torch.logical_and(y_true == lab_i, y_pred == lab_j).float().mean().cpu().numpy()
    return cm

  @property
  def value(self): return self.total/self.count if self.count != 0 else None
  @property
  def name(self):  return 'BinnedConfusionMatrix' 
  
class BinnedNonZeroFalsePositiveRate(BinnedConfusionMatrixMetric):
  """Micro FPR where 0 is negative - (True zeros but predicted nonzero)/(Num true zeros)"""
  @property
  def value(self):
    print('FPR calculation')
    micro_cm = self.total / self.count
    zero_idx = self.labels.index(0.0)
    correctly_predicted_zeros = micro_cm[zero_idx, zero_idx] # TN
    total_true_zeros = micro_cm[zero_idx, :].sum() # N
    TNR = correctly_predicted_zeros/total_true_zeros # TN/N
    # FPR = 1 - TNR
    return correctly_predicted_zeros/total_true_zeros if self.count != 0 else None
  @property
  def name(self):  return 'MicroFPR' 
  
class BinnedTruePositiveRate(BinnedConfusionMatrixMetric):
  """Micro FPR where 0 is negative - (True zeros but predicted nonzero)/(Num true zeros)"""
  def __init__(self, positive_value):
    super().__init__()
    self.positive_value = positive_value
  @property
  def value(self):
    micro_cm = self.total / self.count
    positive_idx = self.labels.index(self.positive_value)
    correctly_predicted = micro_cm[positive_idx, positive_idx] # TP
    total_true = micro_cm[positive_idx, :].sum() # P
    TPR = correctly_predicted/total_true # TP/N
    return TPR if self.count != 0 else None
  @property
  def name(self):  return f'MicroTPR({self.positive_value:.0f})'
  
next_window_metrics = [
  nn.MSELoss(), 
  BinnedNonZeroFalsePositiveRate(), 
  BinnedTruePositiveRate(1.0), 
  BinnedTruePositiveRate(-1.0)
]
next_window_metric_names = [m.name if hasattr(m, 'name') else 'MSE' for m in next_window_metrics]
print(next_window_metrics)
print(next_window_metric_names)

# Segmentation metrics

segmentation_metrics = [
  foreground_acc,
  DiceMulti(),
]
segmentation_metric_names = ['CE', 'UnitAcc', 'Dice']

In [ ]:
def _TestSplitter(max_samples, seed=0):
  "Create function that splits `items` between train/val with `valid_pct` randomly."
  def _inner(o):
      if seed is not None: torch.manual_seed(seed)
      rand_idx = L(list(torch.randperm(len(o)).numpy()))
      cut = min(max_samples, len(o))
      return rand_idx[cut:],rand_idx[:cut]
  return _inner

def test_unit_identification_model(name, batch_size, segment_path, path_to_model):
    
    FASTAI_ARCHES = dict(
      unet_resnet18=resnet18,
      unet_resnet34=resnet34,
      unet_xresnet18=xresnet18_deep,
      unet_xresnet34=xresnet34_deep,
       unet_squeezenet1_0=squeezenet1_0,
      unet_squeezenet1_1=squeezenet1_1,
      unet_densenet121=densenet121,
      unet_densenet169=densenet169,
    )
    
    sc2_segment = DataBlock(
                      blocks=(ImageBlock, MaskBlock(codes=CHANNEL_TO_NAME)),
                      get_items=get_image_files,
                      get_y=lambda filename: (os.path.splitext(filename)[0].replace('images','labels') + '_labels.png'),
                      splitter=_TestSplitter(max_samples=args.max_samples, seed=0),
                      batch_tfms=None)
    dls = sc2_segment.dataloaders(segment_path/'images', shuffle=True, bs=batch_size, num_workers=12)
    print(f'Datset at {segment_path}')
    print(f'Test dataset has approximately {len(dls.valid)*batch_size} windows')

    # Create learner
    learner = unet_learner(arch=FASTAI_ARCHES[name], dls=dls)
    print(f'Loss function for unit id: {learner.loss_func}')
    learner.metrics = segmentation_metrics
    learner = learner.load(path_to_model)    
    
    result = learner.validate()
    # Now that testing has finished, empty the cache
    torch.cuda.empty_cache()
    return {m: r for r, m in zip(result, segmentation_metric_names)}

In [ ]:
print(f'Starting result aggregation on segmentation dataset: {str(segmentation_data_path)}. ')

unit_identification_model_root = extracted_model_root / 'unit_identification'

unit_identification_model_dirs = [f for f in unit_identification_model_root.glob('**/*') 
                                    if f.is_dir() and not str(f).startswith('unet_')]

for model_dir in unit_identification_model_dirs[:min(args.max_models, len(unit_identification_model_dirs))]:
    name = model_dir.name
    arch = name.split('unet_')[-1]
    
    print('\n'*5, f'Starting {name}.'.center(60, '-'))
    
    path_to_model = get_best_model(model_dir)
    
    print(f'Using model {path_to_model}')

    result = test_unit_identification_model(name, args.batch_size, segmentation_data_path, path_to_model)
    
    
    save_benchmark_results(benchmark_results_save_root, result, name, benchmark_kind='unit_identification',
                             sensor_kwargs=None, metric_name='MSE', raw_model_path=path_to_model)
    
    print(f'Finished {name}.')

In [ ]:
pd.read_csv(benchmark_results_save_root/'unit_identification_results.csv')

# Getting *sensor* unit type identification results

In [ ]:
segmentation_dataset = SegmentationDataset(segmentation_data_path,
                                           create_metadata=True)
print(f'Using path: {segmentation_data_path}')

In [ ]:
def test_sensor_unit_identification_model(name, batch_size, segmentation_dataset,
                                          path_to_model, sensor_kwargs):
    
    FASTAI_ARCHES = dict(
      unet_resnet18=resnet18,
      unet_resnet34=resnet34,
      unet_xresnet18=xresnet18_deep,
      unet_xresnet34=xresnet34_deep,
       unet_squeezenet1_0=squeezenet1_0,
      unet_squeezenet1_1=squeezenet1_1,
      unet_densenet121=densenet121,
      unet_densenet169=densenet169,
    )
    
    segmentation_sensor_placement_dataset = SensorPlacementDataset(
            segmentation_dataset, **sensor_kwargs,
            return_mask=False, make_cache=False, noiseless_ground_truth=True)
    
    block = DataBlock(
        blocks=(ImageBlock, MaskBlock),
        get_items=lambda d: list(range(len(d))),
        get_x=lambda idx: segmentation_sensor_placement_dataset[idx][0],
        get_y=lambda idx: segmentation_sensor_placement_dataset[idx][1],
        splitter=_TestSplitter(max_samples=args.max_samples, seed=0),
    )
    
    dls = block.dataloaders(segmentation_sensor_placement_dataset, batch_size=batch_size)
    print(f'Test dataset has {len(segmentation_sensor_placement_dataset)} windows and the valid dataloader has approximately {len(dls.valid)*batch_size} windows')

    # Create learner
    learner = unet_learner(arch=FASTAI_ARCHES[name], dls=dls, n_out=170)
    learner.metrics = segmentation_metrics
    learner = learner.load(path_to_model)    
    
    result = learner.validate()
    # Now that testing has finished, empty the cache
    torch.cuda.empty_cache()
    return {m: r for r, m in zip(result, segmentation_metric_names)}

In [ ]:
print(f'Starting **sensor** result aggregation on segmentation dataset: {str(segmentation_data_path)}. ')

sensor_unit_identification_model_root = extracted_model_root / 'sensor_unit_identification'

sensor_unit_identification_model_dirs = [f for f in sensor_unit_identification_model_root.glob('**/*') 
                                    if f.is_dir() and not str(f).endswith('_sensors')]

for model_dir in sensor_unit_identification_model_dirs[:min(args.max_models, len(sensor_unit_identification_model_dirs))]:
    sensor_kind = model_dir.parent.name.split('_sensors')[0]
    name = model_dir.name
    arch = name.split('unet_')[-1]
    
    sensor_kwargs = {'n_sensors': 50, 'radius': 5.5, 'kind': sensor_kind, 'failure_rate': 0.2}
    
    print('\n'*5, f'Starting {name} with {sensor_kind} placement.'.center(60, '-'))
    
    path_to_model = get_best_model(model_dir)
    
    print(f'Using model {path_to_model}')

    result = test_sensor_unit_identification_model(name, args.batch_size,
                                                   segmentation_dataset, path_to_model, sensor_kwargs)
    
    
    save_benchmark_results(benchmark_results_save_root, result, name, benchmark_kind='unit_identification',
                             sensor_kwargs=sensor_kwargs, metric_name='MSE', raw_model_path=path_to_model)
    
    print(f'Finished {name} with {sensor_kind} placement.')

In [ ]:
pd.read_csv(benchmark_results_save_root/'sensor_unit_identification_results.csv')